## セットアップ

In [ ]:
!pip install -q transformers sentencepiece sentence-transformers \
    torch faiss-cpu datasets scipy scikit-learn numpy

In [ ]:
# VRAMの環境変数設定
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

## モデルの準備

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("pfnet/plamo-embedding-1b", trust_remote_code=True)
model = AutoModel.from_pretrained("pfnet/plamo-embedding-1b", trust_remote_code=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()


## 埋め込み計算モジュールの定義

In [ ]:
import numpy as np

def embed_texts(
    texts: list[str],
    batch_size: int = 32,
    is_query: bool = False,
) -> np.ndarray:
    """
    texts           : 埋め込み対象の文字列リスト
    batch_size      : 一度に投入する文の数（デフォルト：32）
    is_query        : True の場合は model.encode_query を使用
                                False の場合は model.encode_document を使用
    """
    all_embs = []

    with torch.inference_mode():
      # 自動混合精度演算
      with torch.amp.autocast("cuda"):
            for i in range(0, len(texts), batch_size):
                batch = texts[i : i + batch_size]
                if is_query:
                    embs = model.encode_query(batch, tokenizer)
                else:
                    embs = model.encode_document(batch, tokenizer)

                # embs は torch.Tensor
                embs_np = embs.to("cpu").numpy()  # NumPy に変換
                # VRAM を開放する
                del embs
                torch.cuda.empty_cache()
                embs_np = np.nan_to_num(embs_np)
                all_embs.append(embs_np)

    return np.vstack(all_embs)


## JSTSによるモデルの評価

In [ ]:
from datasets import load_dataset

ds = load_dataset("sbintuitions/JMTEB", "jsts")
jsts = ds["test"]
print(jsts.column_names)  # ['sentence_pair_id', 'yjcaptions_id', 'sentence1', 'sentence2', 'label']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# JSTS の全ペアを「文1／文2」と「スコア」に分解
sent1 = jsts["sentence1"]
sent2 = jsts["sentence2"]
gold_score = np.array(jsts["label"], dtype=float)  # 0〜5 の連続値

# 埋め込み
emb1 = embed_texts(sent1)
emb2 = embed_texts(sent2)

# コサイン類似度
# N×2048 の行列同士のコサイン類似度を一度に出す
sim_matrix = cosine_similarity(emb1, emb2)
# 対角成分だけを取り出せばペアごとの類似度が得られる
cos_sim = np.diag(sim_matrix)  # shape=(N,)

In [ ]:
# スピアマンとピアソンの相関係数を計算
from scipy.stats import spearmanr, pearsonr

spearman_corr, _ = spearmanr(cos_sim, gold_score)
pearson_corr, _  = pearsonr(cos_sim, gold_score)

print(f"Spearman: {spearman_corr:.4f}")
print(f"Pearson : {pearson_corr:.4f}")

## インデックス用データ読み込み・前処理・インデックス化

In [11]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


### インデックス化: livedoor-news-corpus

In [ ]:
from datasets import load_dataset
# Livedoor ニュースコーパス (約7,300記事) を train/val/test に分割読み込み
ds = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    shuffle=False,
)
train = ds["train"]
print(train)

In [ ]:
# 検索対象：記事のタイトル＋本文
corpus_texts = [
    f"{row['title']}。{row['content']}"
    for row in train
]

# （例）最初の5件をクエリとして流用
query_texts = corpus_texts[:5]
# 正解ID (今回は自身の記事をトップ1に返す recall@1)
true_ids = list(range(len(query_texts))) # true_ids = [0, 1, 2, ..., 5]

In [ ]:
# 記事のタイトルと本文(corpus_texts)をjsonl形式で保存

import json

with open('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/corpus.jsonl', 'w', encoding='utf-8') as f:
    for text in corpus_texts:
        json.dump({'text': text}, f, ensure_ascii=False)
        f.write('\n')


In [ ]:
# 文を埋め込み化する（時間のかかる処理）
xb = embed_texts(corpus_texts, batch_size=8, is_query=False)  # index 用
xq = embed_texts(query_texts,  batch_size=8, is_query=True)   # query 用

In [ ]:
import faiss

d = xb.shape[1]  # 埋め込み次元
M = 32           # 各点の近傍リンク数
efC = 200        # 構築時の探索深さ

index = faiss.IndexHNSWFlat(d, M)
index.hnsw.efConstruction = efC

# 埋め込みベクトルを追加
index.add(xb)

In [ ]:
index.hnsw.efSearch = 50  # 検索時の探索深さ
k = 5                    # top-k 件取得
D, I = index.search(xq, k) # 検索を実行

In [ ]:
recall1 = np.mean([1 if true_ids[i] in I[i,:1] else 0 for i in range(len(true_ids))])
print(f"Recall@1: {recall1:.3f}")

In [ ]:
# 埋め込みと Faiss インデックスを保存
np.save('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/embeddings_xb.npy', xb)
faiss.write_index(index, '/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/hnsw_index.faiss')

In [ ]:
for i in range(len(query_texts)):
    print(f"Query: {query_texts[i]}")
    for rank, idx in enumerate(I[i]):
        print(f"Top {rank+1}: {corpus_texts[idx]}")
    print("========")

### インデックス化: SFC研究会データ

In [1]:
! git clone https://github.com/kota-yata/Syllabus-proto.git

Cloning into 'Syllabus-proto'...
remote: Enumerating objects: 729, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 729 (delta 149), reused 180 (delta 95), pack-reused 480 (from 1)
Receiving objects: 100% (729/729), 4.47 MiB | 6.02 MiB/s, done.
Resolving deltas: 100% (401/401), done.


In [2]:
import json

filepath = '/content/Syllabus-proto/assets/result-2025s.json'
with open(filepath, 'r', encoding='utf-8') as f:
    all_class_data = json.load(f)

In [4]:
lab_corpus_texts = [] # 埋め込みモデルに入力するテキストのリスト
lab_corpus_metadata = {} # ID をキーとし、メタデータを持つ辞書

# target_subjects の "A", "B" は全角
target_subjects = {"研究会Ａ", "研究会Ｂ"}

for item in all_class_data:
    # --- 必須情報の取得 ---
    sort_id = item.get('sort_id')
    subject_name = item.get('subject_name', '').strip() # 空の場合や前後の空白に対応
    about_text = item.get('about', '').strip()

    # デバッグ用: 取得した subject_name と比較対象を表示
    # print(f"Checking: '{subject_name}' in {target_subjects}")

    # --- subject_name がターゲットに含まれているかチェック ---
    if subject_name not in target_subjects:
        # print(f"Skipping '{subject_name}' - not in target list.") # スキップ理由表示（デバッグ用）
        continue

    # --- ID, subject_name, about がないと意味がないのでスキップ ---
    if not sort_id or not subject_name or not about_text:
        print(f"Skipping item due to missing essential fields: {sort_id or 'Unknown ID'}")
        continue

    # --- 検索対象テキストの生成 ---
    # 研究会名と概要文を結合。改行などで区切ると良い。
    text_for_embedding = f"{subject_name}\n{about_text}"
    lab_corpus_texts.append(text_for_embedding)

    # --- メタデータの抽出・整形 ---
    url = item.get('url', '')
    staff_names = [staff.get('staff_name', '') for staff in item.get('staffs', [])]
    # fields から学部・分野を取得 (リスト形式なので最初のものを取得する例)
    # fields が空リストの場合や、リスト内の辞書が空の場合のエラーを防ぐ
    faculty = ''
    field = ''
    if item.get('fields'): # fields リストが存在するか確認
        first_field = item['fields'][0] if item['fields'] else {} # リストが空でないか確認
        faculty = first_field.get('faculty', '')
        field = first_field.get('field', '')

    term = item.get('term', '')
    # 他に必要なメタデータがあれば追加

    metadata = {
        'subject_name': subject_name,
        'about': about_text, # スニペットは検索時に生成するので、ここでは全文保持
        'url': url,
        'staffs': ", ".join(filter(None, staff_names)), # 教員名をカンマ区切り文字列に
        'faculty': faculty,
        'field': field,
        'term': term
        # 他のメタデータをここに追加
    }
    lab_corpus_metadata[sort_id] = metadata # ID をキーにしてメタデータを保存

print(f"Processed {len(lab_corpus_texts)} items.")
print(f"Total items skipped: {len(all_class_data) - len(lab_corpus_texts)}") # スキップされた数を計算

Skipping item due to missing essential fields: X44849
Processed 165 items.
Total items skipped: 585


In [12]:
# lab_corpus_texts をJSONファイルに保存
corpus_texts_filepath = '/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_corpus_texts.json'
try:
    with open(corpus_texts_filepath, 'w', encoding='utf-8') as f:
        # ensure_ascii=False で日本語がそのまま保存されるようにする
        # indent=4 で整形して見やすくする
        json.dump(lab_corpus_texts, f, ensure_ascii=False, indent=4)
    print(f"Successfully saved corpus texts to {corpus_texts_filepath}")
except IOError as e:
    print(f"Error saving corpus texts: {e}")

# lab_corpus_metadata をJSONファイルに保存
corpus_metadata_filepath = '/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_corpus_metadata.json'
try:
    with open(corpus_metadata_filepath, 'w', encoding='utf-8') as f:
        json.dump(lab_corpus_metadata, f, ensure_ascii=False, indent=4)
    print(f"Successfully saved corpus metadata to {corpus_metadata_filepath}")
except IOError as e:
    print(f"Error saving corpus metadata: {e}")

Successfully saved corpus texts to /content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_corpus_texts.json
Successfully saved corpus metadata to /content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_corpus_metadata.json


## 保存したインデックスを読み込んで使用

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install faiss-cpu
import faiss

### livedoor-news-corpus

In [ ]:
# corpus_textsの読み込み
import json

corpus_texts = []
with open('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/corpus.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        corpus_texts.append(json.loads(line)['text'])

In [ ]:
xb = np.load('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/embeddings_xb.npy')
index = faiss.read_index('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/hnsw_index.faiss')

In [ ]:
query_texts = ["渋谷の事件についてのニュース", "プロ野球の試合結果", "新作映画のレビュー"]
xq = embed_texts(query_texts, is_query=True)

In [ ]:
index.hnsw.efSearch = 50  # 検索時の探索深さ
k = 5  # 上位k件を取得
D, I = index.search(xq, k)  # Dは距離、Iはインデックス

In [ ]:
for i, q in enumerate(query_texts):
  print(f"\n[Query] {q}")
  for j in range(k):
    doc_id = I[i, j] # i番目のクエリにつき5つのID
    print(f" RANK {j+1}: {corpus_texts[doc_id][:50]}...")

## gradioで検索機能を実装

In [ ]:
!pip install -q gradio

In [ ]:
import pandas as pd

# 検索用関数の定義
def search(query: str, k: int = 5, length: int = 30):
    # クエリ埋め込み
    q_emb = embed_texts([query], is_query=True)
    # Faiss 検索
    D, I = index.search(q_emb, k)
    # 結果整形
    results = []
    for score, idx in zip(D[0], I[0]):
        title, snippet = corpus_texts[idx].split("。", 1)  # タイトル・本文を分割
        results.append({"title": title, "snippet": snippet[:length]+"…", "score": float(score)})

    df = pd.DataFrame(results) # gradio用にDFに変換

    return df


In [ ]:
search("パソコン", k=5)

In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=search,
    inputs=[
        gr.Textbox(lines=2, placeholder="検索クエリを入力"),
        gr.Slider(minimum=1, maximum=10, step=1, label="Top k 件数")
    ],
    outputs=gr.Dataframe(
        headers=["title", "snippet", "score"],
        row_count=5
    ),
    title="Livedoor ニュース検索デモ",
    description="pfnet/plamo-embedding-1b + Faiss HNSW による類似ニュース検索"
)

iface.launch(share=True)